# <span style="color:#F72585"><center>Introducción a la Programación Probabilística</center></span>

## <span style="color:#4361EE">¿Qué es?</span> 

La programación probabilística nos ayuda a crear modelos probabilísticos y hacer inferencia a partir de ellos de manera automática. 

## <span style="color:#4361EE">Motivación para y la Programación Probabilística</span> 

1. **Ajustar modelos estadísticos Bayesianos poderosos** 
2. **El Proceso**: *A)Crea un modelo estadístico; B) Corre la inferencia del modelo; C) Evalúa;*
3. **Dificultades con las herramientas existentes para abordar los modelos de interés actuales** 
4. **Usabilidad**: *proósito general, Lenguaje de modelamiento claro, integración*
5. **Escalabilidad**: *Complejidad del modelo, número de parámetros, tamaño de los datos*
6. **Eficiencia**: *Tamaños de muestra altamente efectivos, iteraciones rápidas, baja memoria*
7. **Robustez**: *estructura del modelo (por ejemplo geometría posterior), rutinas numéricas*

## <span style="color:#4361EE">Inferencia. Lo que hay en el transfondo de la Programación Probabilística</span> 

**Hamiltonian Monte Carlo (HMC)**

- *La distribución del parámetro es muestreada en un espacio sin restricciones*. La base teórica es el muestreador Monte Carlo Hamiltoniano, que permite escanear el espacio del parámetro de manera mucho más eficiente que lo que hacen los muestreadores clásicos Metrópolis- Hastings o Gibbs y sus derivados.
- *Transformaciones del parámetro*: Ajuste automático del Jacobiano asociado a las  transformaciones.
- *Diferenciación automática*: Gradientes del modelo con respecto a los parámetros usando diferenciación automática.

**No-U-Turn Sampler (NUTS)**

- *Ajuste (tunning)* automático de los parámetros del muestreador. 
- *Calentamiento*: Estimación de la matriz de masa y tamaño del paso del muestreador.
- *Muestreo*: Número adaptativo de pasos.
- *Mantiene el balance detallado*. Que asegura tener cadenas de Markov reversibles.

**Optimización: estimación modal**

- Métodos BFGS y Newton.

## <span style="color:#4361EE">Para científicos y más...</span> 


- Lenguaje probabilístico flexible, el lenguaje sigue creciendo

- Enfoque en la ciencia: el modelado y los supuestos.

- Acceso a múltiples algoritmos (el valor predeterminado es bastante bueno)

- Más rápido y menos propenso a errores que la implementación desde cero

- Implementación eficiente

- Mucha ayuda de modelado (gratis) en la lista de usuarios

- Desarrolladores receptivos, soporte continuo para Pyro

- No solo por inferencia

- Muestreo de avance rápido; muchas distribuciones 

- Gradientes para funciones arbitrarias

## <span style="color:#4361EE">Nuestras Herramientas</span> 


### <span style="color:#4CC9F0">Pyro</span>


Pyro es un lenguaje de programación probabilística de alto nivel escrito en Python y compilada sobre [PyTorch](https://pytorch.org/) que combina el estado del arte de técnicas de IA con la experiencia y habilidades de científicos de datos para unificar el Aprendizaje Profundo con el modelamiento Bayesiano. El usuario puede especificar o crear modelos estadísticos y hacer Inferencia Bayesiana a partir de ellos. Si no tiene familiaridad con [PyTorch](https://pytorch.org/) le recomendamos que se familiarice con el lenguaje por medio del tutorial [learn the basics](https://pytorch.org/tutorials/beginner/basics/intro.html).


- Homepage: https://pyro.ai/
- Documentation: https://docs.pyro.ai/en/stable/
- Forum: https://forum.pyro.ai/

### <span style="color:#4CC9F0">TensorFlow Probability (TFP)</span>


Es una biblioteca de Python compilada sobre [TensorFlow](https://www.tensorflow.org/) para la creación y combinación de modelos probabilísticos con el fin de comprender los datos y hacer predicciones. Incluye optimizadores como Nelder-Mead, BFGS y SGLD. Si no tiene familiaridad con [TensorFlow](https://www.tensorflow.org/) le recomendamos que se familiarice con el lenguaje por medio de los tutoriales disponibles en [Introducción a TensorFlow](https://www.tensorflow.org/learn)

### <span style="color:#4CC9F0">Stan</span>



Stan es un lenguaje de programación probabilística de alto nivel, que permite al usuario especificar y crear modelos estadísticos de manera sencilla.
**Interfaces de usuario para distintas plataformas**: 

- CmdStan - command line
- RStan - Integración con R
- PyStan - Integración con Python
- MStan - Integración con Matlab (user contributed)

1.  User Guide: https://mc-stan.org/users/documentation/
2.  Homepage: http://mc-stan.org
3.  Stan Users group: https://groups.google.com/d/forum/stan-users


## <span style="color:#4361EE">Un primer ejemplo</span> 


Introducimos un primer ejemplo general. El desarrollo completo de este ejemplo puede encontrarse en [Models in Pyro](https://pyro.ai/examples/intro_long.html)

### <span style="color:#4CC9F0">Modelo Estadístico</span>


Supongamos que tenemos observaciones $[y_n |x_n], n=1,\ldots,N$, y asumamos el modelo Bayesiano

$$
\begin{align*}
y_n &\sim \mathcal{N}(\alpha + \beta x_n, \sigma^2),\quad n=1,\ldots,N\\
\alpha &\sim \mathcal{N}(0, 100)\\
\beta &\sim \mathcal{N}(0, 100)\\
\sigma &\sim \mathcal{C}auchy(0, 25) 1_{\sigma>0}\\
\end{align*}
$$


````{tab} Pyro
```python
def model(x, y=None):
    alpha = pyro.sample("alpha",dist.Normal(0,10))
    beta = pyro.sample("beta",dist.Normal(0,10))
    sigma = pyro.sample("sigma",dist.Cauchy(0,25),constraint=constraints.positive)
    mu = alpha + beta * x
    with pyro.plate("plate"):
        pyro.sample("y",dist.Normal(alpha+beta*x,sigma),obs=y )
```
````
````{tab} TFP
```python
def model(x):
    return tfd.JointDistributionSequential([
        tfd.Sample(tfd.Normal(loc=0., scale=10., name="alpha"), sample_shape=1), #alpha
        tfd.Sample(tfd.Normal(loc=0., scale=10., name='beta'), sample_shape=1), #beta
        tfd.Sample(tfd.HalfCauchy(loc=0., scale=25., name='sigma'), sample_shape=1), #sigma  
        lambda alpha,beta,sigma,x: (
            tfd.Normal(loc= alpha+beta*x, scale=sigma, name='y')
        )  
    ])
```
````
````{tab} Stan
```r
data { 
int<lower=0> N;
vector[N] y;
vector[N] x;
}


parameters {
real alpha;
real beta;
real<lower=0> sigma;
}


model {
alpha ~ normal(0,10);
beta ~ normal(0,10);
sigma ~ cauchy(0,5);
for (n in 1:N)
   y[n] ~ normal(alpha + beta * x[n], sigma);
}
```
````